## Bibliotecas

In [49]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from tqdm import tqdm

## Transforms (data augmentation online para o treino)

In [ ]:
IMG_SIZE = 224 # trabalha com imagens 224x224 com 3 canais RGB, as transforms colocarão as imagens nesse padrão

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #normaliza os canais R, G e B da imagem para a ResNet50 ImageNet
                         std=[0.229, 0.224, 0.225]),
])

## Datasets e DataLoaders

In [51]:
from PIL import Image, UnidentifiedImageError

def safe_loader(path):
    try:
        return Image.open(path).convert("RGB")
    except UnidentifiedImageError:
        print(f"[ERRO] Imagem corrompida ignorada: {path}")
        # retorna uma imagem branca 224x224 (não quebra o DataLoader)
        return Image.new("RGB", (224, 224), (255, 255, 255))

In [ ]:
data_dir = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/splits"

train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform, loader=safe_loader) # shuffle evita aprender ordem
val_ds   = datasets.ImageFolder(os.path.join(data_dir, "val"),   transform=val_transform,   loader=safe_loader)
test_ds  = datasets.ImageFolder(os.path.join(data_dir, "test"),  transform=val_transform,  loader=safe_loader)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4)

num_classes = len(train_ds.classes)
print("Total de classes:", num_classes)

Total de classes: 30


## Carregar a ResNet50 pré-treinada

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carrega modelo ResNet50 já treinado com ImageNet (já sabefeatures gerais de imagens como bordas, textura, etc.)
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Treina todas as camadas sem congelamento
for param in model.parameters():
    param.requires_grad = True 

# Substituir a última camada de 1000 classes (ImageNet) para 30 do nosso problema (fine tunning)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

## Otimizador, Loss e Scheduler

In [ ]:
criterion = nn.CrossEntropyLoss() # Função de Perda (Loss)

optimizer = optim.Adam(model.parameters(), lr=1e-4) # Otimizador baseado em gradiente

scheduler = optim.lr_scheduler.ReduceLROnPlateau( # ajusta a taxa de aprendizado se o progresso travar
    optimizer, factor=0.5, patience=3
)

## Funções de treino e validação

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total, correct, total_loss = 0, 0, 0

    # itera sob s batches (conjunto de imagens)
    for imgs, labels in tqdm(loader, desc="Treinando"):
        imgs, labels = imgs.to(device), labels.to(device) #carrega na cpu/gpu

        optimizer.zero_grad() # reseta o gradiente
        outputs = model(imgs) # obtém as classificações
        loss = criterion(outputs, labels) # obtém as predições comparando com os labels e calcula o loss
        
        loss.backward() # calcula o gradiente da loss em relação aos pesos 
        optimizer.step() # atualiza os pesos com base no gradiente

        # Métricas
        total_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return total_loss / total, correct / total


def validate(model, loader, criterion):
    model.eval()
    total, correct, total_loss = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validando"):
            imgs, labels = imgs.to(device), labels.to(device) # carrega
            outputs = model(imgs) # avalia
            loss = criterion(outputs, labels) # calcula a loss

            # Métricas
            total_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += imgs.size(0)

    return total_loss / total, correct / total

## Loop de treinamento com Early Stopping

In [56]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce GTX 1650


In [ ]:
EPOCHS = 1
best_val_acc = 0
patience = 6
wait = 0

for epoch in range(EPOCHS):
    print(f"\n===== Época {epoch+1}/{EPOCHS} =====")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Treino  Loss: {train_loss:.4f}  Acc: {train_acc:.4f}")
    print(f"Validação Loss: {val_loss:.4f}  Acc: {val_acc:.4f}")

    scheduler.step(val_loss)

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "best_resnet50.pth")
        print("Modelo salvo (melhor até agora)")
    else:
        wait += 1
        print(f"Early stopping counter: {wait}/{patience}")

    if wait >= patience:
        print("Early stopping ativado")
        break


===== Época 1/1 =====


Validando: 100%|██████████| 35/35 [00:06<00:00,  5.51it/s]


Treino  Loss: 1.1291  Acc: 0.6763
Validação Loss: 0.4303  Acc: 0.8743
✔ Modelo salvo (melhor até agora)


## Avaliar no conjunto de teste

In [58]:
model.load_state_dict(torch.load("best_resnet50.pth"))
test_loss, test_acc = validate(model, test_loader, criterion)
print("\nRESULTADO FINAL NO TESTE:")
print(f"Acurácia: {test_acc:.4f}")

Validando: 100%|██████████| 36/36 [00:06<00:00,  5.47it/s]


RESULTADO FINAL NO TESTE:
Acurácia: 0.8784


## Avaliar um classe específica do conjunto de testes

In [ ]:
from torch.utils.data import Subset


classe_escolhida = "Gobio gobio"

# pega o índice da classe
idx_classe = test_ds.class_to_idx[classe_escolhida]
print("[INFO] Classe:", classe_escolhida, "-> índice:", idx_classe)

# filtra apenas imagens dessa classe
indices = [i for i, (_, lab) in enumerate(test_ds.samples) if lab == idx_classe]

# cria um subset
subset_classe = Subset(test_ds, indices)

# loader só dessa classe
test_loader_classe = torch.utils.data.DataLoader(
    subset_classe, batch_size=32, shuffle=False, num_workers=4
)

# avaliação
model.load_state_dict(torch.load("best_resnet50.pth"))
loss, acc = validate(model, test_loader_classe, criterion)

print(f"\n=== RESULTADOS PARA A CLASSE '{classe_escolhida}' ===")
print(f"Acurácia: {acc:.4f}")


[INFO] Classe: Gobio gobio -> índice: 12


Validando: 100%|██████████| 2/2 [00:00<00:00,  3.81it/s]


=== RESULTADOS PARA A CLASSE 'Gobio gobio' ===
Acurácia: 0.8286
